

Please Upvote if you like my work.


# Phishing Websites Classifications




#### Abstract:

The phishing website has evolved as a major cybersecurity threat in recent times. The phishing
websites host spam, malware, ransomware, drive-by exploits, etc. A phishing website many a time
look-alike a very popular website and lure an unsuspecting user to fall victim to the trap. The
victim of the scams incurs a monetary loss, loss of private information and loss of reputation.

Hence, it is imperative to find a solution that could mitigate such security threats in a timely
manner. Traditionally, the detection of phishing websites is done using blacklists. There are many
popular websites which host a list of blacklisted websites, e. g. PhisTank. The blacklisting technique
lack in two aspects, blacklists might not be exhaustive and do not detect a newly generated phishing 
website. In recent times machine learning techniques have been used in the classification and 
detection of phishing websites. 

Here ,we will build some models to classifiy if  the website is a phishinf website or not , using 
various models . 


### Dataset

- Domain: The URL itself.
- Ranking: Page Ranking
- isIp: Is there an IP address in the weblink
- valid: This data is fetched from google's whois API that tells us more about the current     status of the URL's registration.
- activeDuration: Also from whois API. Gives the duration of the time since the registration   up until now.
- urlLen: It is simply the length of the URL
- is@: If the link has a '@' character then it's value = 1
- isredirect: If the link has double dashes, there is a chance that it is a redirect. 1->     multiple dashes present together.
- haveDash: If there are any dashes in the domain name.
- domainLen: The length of just the domain name.
- noOfSubdomain: The number of subdomains preset in the URL.
- Labels: 0 -> Legitimate website , 1 -> Phishing Link/ Spam Link

In [ ]:
#This Python3 environment comes with many helpful analytics libraries installed

#It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
#For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Input data files are available in the read-only "../input/" directory
#For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
#You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# importing all the packages

%matplotlib inline
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn 
import seaborn as sns
from random import random 
import scipy.stats as stats
import math


import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_absolute_error,classification_report,confusion_matrix
from pandas_profiling import ProfileReport

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV


import warnings
warnings.filterwarnings("ignore")


In [ ]:
#importing the datadet 
df=pd.read_csv('/kaggle/input/phishing-data/combined_dataset.csv')

#### Let's check the dataset 

In [ ]:
df.head()

In [ ]:
df.shape


In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
# we are adding here an column named 'ID'  
df['ID'] = np.arange(len(df))

#arranging that column  
cols = list(df.columns)
cols = [cols[-1]] + cols[:-1]
df = df[cols]

In [ ]:
df.head()

In [ ]:
# Profile Report is very amazing function and it display detailed information of every
# column of the dataset

ProfileReport(df,minimal=True)

In [ ]:
df.columns

In [ ]:
unique_domain=df.domain.unique()
len(unique_domain)

#### Potting various attributes and comparing with each other 

In [ ]:
df.plot(kind='scatter',x='ID',y='ranking')
plt.show()

In [ ]:
sns.jointplot(data=df,x='ID',y='ranking')

In [ ]:
sns.catplot(x="nosOfSubdomain", y="domainLen", data=df )
plt.show()

In [ ]:
sns.countplot(df['isIp'],color='r')

In [ ]:
sns.barplot(x="valid", y="domainLen", data=df)

In [ ]:
sns.lineplot(y=df.urlLen,x=df.domainLen,data=df)

In [ ]:
fig=px.pie(df,df['isIp'],width=500,height=500)
fig.show()

In [ ]:
fig=px.pie(df,df['valid'])
fig.show()

In [ ]:
corr=df.corr()
sns.heatmap(corr)

In [ ]:
df_tmp=df.copy()

In [ ]:
# convert th estring object to category 
for label,content in  df.items():
    if pd.api.types.is_string_dtype(content):
        df_tmp[label]=content.astype('category').cat.as_ordered()

In [ ]:
# our dataset now contains no string type datatype
df_tmp.info()

In [ ]:
#convert category to int data type
for label,content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        df_tmp[label]=pd.Categorical(content).codes+1

In [ ]:
#now all data is in int data types
df_tmp.info()

In [ ]:
#also our dataset contains zero missing vaules , so we can now proceed with modelling
df_tmp.isna().sum()

## MODELLING OF THE DATASET 

In [ ]:
# splitting the dataset in x ,and y.

x=df_tmp.drop('label',axis=1)
y=df_tmp['label']

In [ ]:
# using the linear regression model. 
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

np.random.seed(42)
linear_model=LinearSVC()
linear_model.fit(x_train,y_train)

In [ ]:
np.random.seed(42)
linear_model.score(x_test,y_test)

Now let's see if we can improve our model score and accuracy 
by using various models and tuning hyperparameters

In [ ]:
np.random.seed(42)
model=RandomForestClassifier(n_jobs=1)
model.fit(x_train,y_train)

In [ ]:
np.random.seed(42)
model_score=model.score(x_test,y_test)
model_score

### Randomized Search CV and Grid Search CV

In [ ]:
np.random.seed(42)

grid={'classifier__n_estimators':[10,100,200,500],
      'classifier__max_depth':[None,5,10,50,100],
      'classifier__max_features':['auto','sqrt'],
      'classifier__max_sample_split':[2,5,10],
      'classifier__min_sample_leaf':[1,5,10]
}

### K-Nearest Neighbors Classifier

In [ ]:
# Trying K-NN Classifier model  
np.random.seed(42)
neigh=KNeighborsClassifier()
neigh.fit(x_train,y_train)

In [ ]:
np.random.seed(42)
neigh_score=neigh.score(x_test,y_test)
neigh_score

In [ ]:
grid_neigh={'n_neighbors':[1,5],
    'weights':['uniform','distance'],
    'algorithm':['auto'],
    'leaf_size':[10,30],
    'p':[3,7],
    
   
    }

In [ ]:
rs_neigh=RandomizedSearchCV(estimator=neigh,
                           param_distributions=grid_neigh,
#                            n_iter=10,
                           cv=5,
                           verbose=2)

rs_neigh.fit(x_train,y_train)

In [ ]:
rs_neigh_score=rs_neigh.score(x_test,y_test)
rs_neigh_score

In [ ]:
rs_neigh_grid=GridSearchCV(estimator=neigh,
                           param_grid=grid_neigh,
                           cv=5,
                           verbose=2
                           )
rs_neigh_grid.fit(x_train,y_train)

In [ ]:
rs_neigh_grid=rs_neigh_grid.score(x_test,y_test)
rs_neigh_grid

### Naive Bayes

In [ ]:
np.random.seed(42)
nb_model=GaussianNB()
nb_model.fit(x_train,y_train)
pred_nb=nb_model.predict(x_test)
nb_model1=nb_model.score(x_test,y_test)
nb_model1

In [ ]:
np.random.seed(42)
print(accuracy_score(pred_nb,y_test))
print(confusion_matrix(pred_nb,y_test))
print(classification_report(pred_nb,y_test))

### So we have tried and tested few models and algorithms to get best results: accuracy_score ,etc. and we found that RandomForestClassifier is the best model we found !!!

### You can also tune the hyperparameters for better results ..